In [1]:

# GBDT
from sklearn.ensemble import GradientBoostingRegressor
# XGBoost
import xgboost as xgb
# LightGBM
import lightgbm as lgb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import pickle
import multiprocessing
from sklearn.preprocessing import StandardScaler
ss = StandardScaler() 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,LinearRegression,LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest

In [10]:
train_path = '/Users/gongbin/Documents/数据竞赛/Datawhale/team-learning/数据竞赛（房租预测）/Data/train_data_clean.csv'
test_path = '/Users/gongbin/Documents/数据竞赛/Datawhale/team-learning/数据竞赛（房租预测）/Data/test_a_clean.csv'
test_y_path = '/Users/gongbin/Documents/数据竞赛/Datawhale/team-learning/数据竞赛（房租预测）/Data/评分文件/sub_a_913.csv'
train_X = pd.read_csv(train_path)
test_X = pd.read_csv(test_path)
train_X.pop('Type')
test_X.pop('Type')
train_X.pop('Unnamed: 0')
test_X.pop('Unnamed: 0')
test_Y = pd.read_csv(test_y_path)
train_Y = train_X.pop('tradeMoney')


##  1.使用RandomForestRegressor训练模型


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

randomForest_model = RandomForestRegressor()
kf = KFold(n_splits=5, shuffle=True)
score_ndarray = cross_val_score(randomForest_model, train_X, train_Y, cv=kf)
print(score_ndarray)
print(score_ndarray.mean())

/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/gongbin/opt/a

[0.8698089  0.86056329 0.86161657 0.86850468 0.87583451]
0.8672655891715111


In [15]:
from sklearn.metrics import r2_score
sample_leaf_options = [10,50,100,200,500]

max_score = 0
for leaf_size in sample_leaf_options:
    model = RandomForestRegressor(n_estimators = 200, n_jobs = -1,random_state =50,
                                  max_features = "auto", min_samples_leaf = leaf_size)
    model.fit(train_X, train_Y)
    score = r2_score(model.predict(test_X), test_Y)
    if  score > max_score:
        max_score = score
        print('max score:' , max_score)
        best_model = model
        #score
    
 


max score: 0.8931630753825089


In [16]:
model_rf = best_model  #保存最佳的rf模型

## 训练lgb模型

In [17]:
lgb_train = lgb.Dataset(train_X, train_Y)
lgb_eval = lgb.Dataset(test_X, test_Y, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)



print('Starting predicting...')
# predict
pred_Y = gbm.predict(test_X, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', r2_score(test_Y, pred_Y))


Starting training...
[1]	valid_0's l1: 1621.28	valid_0's l2: 5.64774e+06
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 1558.16	valid_0's l2: 5.22935e+06
[3]	valid_0's l1: 1500.42	valid_0's l2: 4.85181e+06
[4]	valid_0's l1: 1443.98	valid_0's l2: 4.50418e+06
[5]	valid_0's l1: 1393.9	valid_0's l2: 4.19316e+06
[6]	valid_0's l1: 1339.11	valid_0's l2: 3.88831e+06
[7]	valid_0's l1: 1287.43	valid_0's l2: 3.60272e+06
[8]	valid_0's l1: 1238.35	valid_0's l2: 3.34518e+06
[9]	valid_0's l1: 1192.87	valid_0's l2: 3.11375e+06
[10]	valid_0's l1: 1152.64	valid_0's l2: 2.91482e+06
[11]	valid_0's l1: 1114.88	valid_0's l2: 2.72739e+06
[12]	valid_0's l1: 1077.15	valid_0's l2: 2.54405e+06
[13]	valid_0's l1: 1043	valid_0's l2: 2.38106e+06
[14]	valid_0's l1: 1009.74	valid_0's l2: 2.2386e+06
[15]	valid_0's l1: 979.876	valid_0's l2: 2.11532e+06
[16]	valid_0's l1: 950.591	valid_0's l2: 1.98549e+06
[17]	valid_0's l1: 923.949	valid_0's l2: 1.87762e+06
[18]	valid_0's l1: 896.471	valid

## 对模型进行融合


In [19]:
train_X.columns

Index(['area', 'rentType', 'houseFloor', 'totalFloor', 'houseToward',
       'houseDecoration', 'communityName', 'region', 'plate', 'buildYear',
       'saleSecHouseNum', 'totalTradeMoney', 'totalTradeArea',
       'tradeMeanPrice', 'tradeSecNum', 'totalNewTradeMoney',
       'totalNewTradeArea', 'tradeNewMeanPrice', 'tradeNewNum', 'remainNewNum',
       'supplyNewNum', 'supplyLandNum', 'supplyLandArea', 'tradeLandNum',
       'tradeLandArea', 'landTotalPrice', 'landMeanPrice', 'totalWorkers',
       'newWorkers', 'residentPopulation', 'pv', 'uv', 'lookNum', 'month',
       'year', 'Room', 'Hall', 'Bath', 'Room_Bath', 'trainsportNum',
       'all_SchoolNum', 'all_hospitalNum', 'all_mall', 'otherNum'],
      dtype='object')

In [20]:
train_X['communityName']

0          50
1         129
2         178
3         312
4        1256
         ... 
40210    1208
40211     852
40212     851
40213     790
40214    3245
Name: communityName, Length: 40215, dtype: int64

In [29]:
categorical_feats = ['rentType', 'houseFloor', 'houseToward', 'houseDecoration',  'region', 'plate','communityName']

In [30]:
train_X[categorical_feats].head()

,rentType,houseFloor,houseToward,houseDecoration,region,plate,communityName
0,2,1,6,0,0,63,50
1,2,0,6,2,1,48,129
2,2,1,6,0,1,49,178
3,2,0,6,3,1,50,312
4,2,1,6,1,2,43,1256


In [37]:
lgb_params={
    'objective':'regression',
    #'booster':'gbtree',
    #'num_class':10, # 类别数，与 multisoftmax 并用
    'gamma':0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth':12, # 构建树的深度，越大越容易过拟合
    'lambda':2,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample':0.7, # 随机采样训练样本
    'colsample_bytree':0.7, # 生成树时进行的列采样
    'min_child_weight':3, 
# 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
#，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
#这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
    'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007, # 如同学习率
    'seed':1000,
    'nthread':7,# cpu 线程数
    'eval_metric': 'l2'
}

In [43]:
def get_r2_metric(preds, label):
    return ('r2', r2_score(preds, label), True)

In [52]:
from sklearn.model_selection import KFold

def model_ensemble(feature = train_X, label = train_Y, categorical_feats = categorical_feats, params = lgb_params,
                  test = test_X):
    folds = KFold(n_splits=5, shuffle=True, random_state=2333)
   
    "===================================第一轮========================================================"
    y_pre_list = []
    r2_list = []
    train_feat = pd.Series()
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(feature.values, label)):
        print("fold {}".format(fold_))
        trn_data = lgb.Dataset(feature.iloc[trn_idx], label[trn_idx], categorical_feature=categorical_feats)
        val_data = lgb.Dataset(feature.iloc[val_idx], label[val_idx], categorical_feature=categorical_feats)

        num_round = 5000
        clf = lgb.train(params, trn_data, num_round,valid_sets=[trn_data, val_data], verbose_eval=500,
                    early_stopping_rounds=200)
        y_pre = clf.predict(feature.iloc[val_idx], num_iteration=clf.best_iteration)
        r2 = r2_score(y_pre,label[val_idx])
        r2_list.append(r2)
        train_feat = train_feat.append(pd.Series(y_pre,index=val_idx))
        y_pre_test = clf.predict(test,num_iteration=clf.best_iteration)
        y_pre_list.append(y_pre_test)
    print('r2 score{:}'.format(r2))
    print('r2:{:}'.format(np.mean(r2_list)))

    y_pred_final=  (y_pre_list[0]+y_pre_list[1]+y_pre_list[2]+y_pre_list[3]+y_pre_list[4])/5
    feature['pre'] = train_feat
    test['pre'] = y_pred_final
    "===================================第二轮========================================================"
    y_pre_list = []
    r2_list = []
    train_feat = pd.Series()
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(feature.values, label)):
        print("fold {}".format(fold_))
        trn_data = lgb.Dataset(feature.iloc[trn_idx], label[trn_idx], categorical_feature=categorical_feats)
        val_data = lgb.Dataset(feature.iloc[val_idx], label[val_idx], categorical_feature=categorical_feats)

        num_round = 5000
        clf = lgb.train(params, trn_data, num_round, valid_sets=[trn_data, val_data], verbose_eval=500,
                    early_stopping_rounds=200)
        y_pre = clf.predict(feature.iloc[val_idx], num_iteration=clf.best_iteration)
        r2 = r2_score(y_pre,label[val_idx])
        r2_list.append(r2)
        train_feat = train_feat.append(pd.Series(y_pre,index=val_idx))
        y_pre_test = clf.predict(test,num_iteration=clf.best_iteration)
        y_pre_list.append(y_pre_test)
    print('r2 score{:}'.format(r2))
    print('r2:{:}'.format(np.mean(r2_list)))
    
    y_pred_final=  (y_pre_list[0]+y_pre_list[1]+y_pre_list[2]+y_pre_list[3]+y_pre_list[4])/5
    feature['pre_2'] = train_feat
    test['pre_2'] = y_pred_final
    "=======================第三轮========================================================"
    y_pre_list = []
    r2_list = []
    train_feat = pd.Series()
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(feature.values, label)):
        print("fold {}".format(fold_))
        trn_data = lgb.Dataset(feature.iloc[trn_idx], label[trn_idx], categorical_feature=categorical_feats)
        val_data = lgb.Dataset(feature.iloc[val_idx], label[val_idx], categorical_feature=categorical_feats)

        num_round = 5000
        clf = lgb.train(params, trn_data, num_round, valid_sets=[trn_data, val_data], verbose_eval=500,
                    early_stopping_rounds=200)
        y_pre = clf.predict(feature.iloc[val_idx], num_iteration=clf.best_iteration)
        r2 = r2_score(y_pre,label[val_idx])
        r2_list.append(r2)
        train_feat = train_feat.append(pd.Series(y_pre,index=val_idx))
        y_pre_test = clf.predict(test,num_iteration=clf.best_iteration)
        y_pre_list.append(y_pre_test)
    print('r2 score{:}'.format(r2))
    print('r2:{:}'.format(np.mean(r2_list)))
    
    y_pred_final=  (y_pre_list[0]+y_pre_list[1]+y_pre_list[2]+y_pre_list[3]+y_pre_list[4])/5
    
    return y_pred_final



In [53]:
y_pred = model_ensemble()

fold 0
Training until validation scores don't improve for 200 rounds
[500]	training's l2: 605643	valid_1's l2: 730697
Early stopping, best iteration is:
[505]	training's l2: 604089	valid_1's l2: 730524
fold 1


/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


Training until validation scores don't improve for 200 rounds
[500]	training's l2: 608310	valid_1's l2: 726770
Early stopping, best iteration is:
[575]	training's l2: 586813	valid_1's l2: 725996
fold 2


/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


Training until validation scores don't improve for 200 rounds
[500]	training's l2: 610466	valid_1's l2: 735016
Early stopping, best iteration is:
[666]	training's l2: 568757	valid_1's l2: 730433
fold 3
Training until validation scores don't improve for 200 rounds


...

Training until validation scores don't improve for 200 rounds
[500]	training's l2: 622746	valid_1's l2: 749471
Early stopping, best iteration is:
[590]	training's l2: 595472	valid_1's l2: 747401
fold 3


/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


Training until validation scores don't improve for 200 rounds
[500]	training's l2: 628211	valid_1's l2: 719485
Early stopping, best iteration is:
[549]	training's l2: 611743	valid_1's l2: 718420
fold 4


/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/gongbin/opt/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


Training until validation scores don't improve for 200 rounds
[500]	training's l2: 603195	valid_1's l2: 850652
[1000]	training's l2: 507505	valid_1's l2: 845126
[1500]	training's l2: 452503	valid_1's l2: 833973
[2000]	training's l2: 412896	valid_1's l2: 821673
[2500]	training's l2: 380918	valid_1's l2: 812009
[3000]	training's l2: 355098	valid_1's l2: 804203
[3500]	training's l2: 333014	valid_1's l2: 797672
[4000]	training's l2: 314035	valid_1's l2: 791509
[4500]	training's l2: 297455	valid_1's l2: 785641
[5000]	training's l2: 283009	valid_1's l2: 780737
Did not meet early stopping. Best iteration is:
[5000]	training's l2: 283009	valid_1's l2: 780737
r2 score0.8669764564933371
r2:0.8619147984769511
